In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from googleapiclient.discovery import build
import csv
import re
import datetime
from kaggle_secrets import UserSecretsClient

In [2]:
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("YouTubeAPIKey")
youtube = build('youtube', 'v3', developerKey=api_key)

In [3]:
def parse_duration(duration):
    """
    Parses the ISO 8601 duration string (e.g., PT14M22S) into total seconds.
    """
    minutes = 0
    seconds = 0

    minutes_match = re.search(r'(\d+)M', duration)
    if minutes_match:
        minutes = int(minutes_match.group(1))

    seconds_match = re.search(r'(\d+)S', duration)
    if seconds_match:
        seconds = int(seconds_match.group(1))

    # Convert everything to seconds
    total_seconds = minutes * 60 + seconds
    return total_seconds

In [4]:
def get_video_details(video_ids):
    """
    Fetch detailed video information, including likes, dislikes, comment count, videoId, publishedAt, and channelId.
    """
    request = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        id=','.join(video_ids)
    )
    response = request.execute()

    video_details = []
    for item in response['items']:
        video_data = {
            'videoId': item['id'],  # Adding videoId
            'publishedAt': item['snippet']['publishedAt'],  # Adding publishedAt
            'channelId': item['snippet']['channelId'],  # Adding channelId
            'title': item['snippet']['title'],
            'description': item['snippet']['description'],
            'video_url': f"https://www.youtube.com/watch?v={item['id']}",
            'likes': item['statistics'].get('likeCount', 'N/A'),
            'dislikes': item['statistics'].get('dislikeCount', 'N/A'),
            'comment_count': item['statistics'].get('commentCount', 'N/A'),
        }
        video_details.append(video_data)

    return video_details


In [5]:
def get_trending_food_videos():
    """
    Fetch top 50 trending food videos based on a query and filter out shorts (duration <= 60 seconds).
    """
    query = 'healthy meals OR quick recipes OR plant-based meals OR meal prep OR vegan recipes OR fitness meals'

    # Get video data based on the search query
    request = youtube.search().list(
        q=query,  # Broad query for food-related videos
        part='snippet',
        type='video',
        order='viewCount',  # Sort by view count to get popular videos
        maxResults=50  # Get top 50 results
    )
    response = request.execute()

    video_ids = []
    for item in response['items']:
        video_ids.append(item['id']['videoId'])

    # Fetch detailed information (likes, dislikes, comments)
    videos = get_video_details(video_ids)

    # Filter out Shorts (duration <= 60 seconds)
    filtered_videos = []
    for video in videos:
        video_id = video['video_url'].split('v=')[1]
        request = youtube.videos().list(
            part='contentDetails',
            id=video_id
        )
        video_duration_response = request.execute()
        duration = video_duration_response['items'][0]['contentDetails']['duration']

        # Parse the duration string and check if it's longer than 60 seconds
        duration_seconds = parse_duration(duration)
        if duration_seconds > 60:
            filtered_videos.append(video)

    return filtered_videos


In [6]:
def get_trending_shorts():
    """
    Fetch top 100 trending shorts related to food.
    """
    query = 'healthy meals OR quick recipes OR plant-based meals OR meal prep OR vegan recipes OR fitness meals'

    # Get top 100 trending shorts related to food
    request = youtube.search().list(
        q=query,  # Search query
        part='snippet',
        type='video',
        videoDuration='short',  # Filters for videos with duration less than 60 seconds (Shorts)
        order='viewCount',  # Sort by view count to get popular shorts
        maxResults=50  # Get top 50 results
    )
    response = request.execute()

    video_ids = []
    for item in response['items']:
        video_ids.append(item['id']['videoId'])

    # Fetch detailed information (likes, dislikes, comments)
    shorts = get_video_details(video_ids)

    return shorts


In [7]:
def save_data_to_csv(data, filename):
    """
    Save video data to a CSV file, including videoId, publishedAt, and channelId.
    """
    # Updated headers to include 'videoId', 'publishedAt', and 'channelId'
    headers = ['videoId', 'publishedAt', 'channelId', 'title', 'description', 'video_url', 'likes', 'dislikes', 'comment_count']
    
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        
        # Write each video data to the CSV with the new fields
        for item in data:
            writer.writerow(item)

In [8]:
def send_email(subject, body):
    """
    Send an email notification after the script runs successfully.
    """
    sender_email = "darshanabk1112@gmail.com"
    receiver_email = "darshanabk1112@gmail.com"
    password = "gcae fzim tvzd ibaj"  # Use environment variables for security

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    message.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message.as_string())
        print("Email sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

In [9]:
def main():
    # Get the current week number for naming the file
    TimeStamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Get top 50 trending food videos
    videos = get_trending_food_videos()
    # print("Top 50 Trending Food Videos:")
    # for video in videos:
    #     print(video)

    # Get top 50 trending Shorts
    shorts = get_trending_shorts()
    # print("\nTop 50 Trending Shorts:")
    # for short in shorts:
    #     print(short)

    # Save data to CSV
    save_data_to_csv(videos, f'top_50_trending_food_videos_{TimeStamp}.csv')
    save_data_to_csv(shorts, f'top_50_trending_food_shorts_{TimeStamp}.csv')

    # Send email notification after successful completion
    send_email(
        "Weekly Trending Food Videos and Shorts Update",
        f"The weekly update for the top trending food videos and shorts has been successfully fetched and saved."
    )


In [10]:
if __name__ == "__main__":
    main()

Email sent successfully.
